# CNZS Pilot Test Survey – Long Format Transformation
This notebook transposes survey responses into long format with question metadata.

## Step 0: Setup & Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 150)

In [2]:
# ============================================================
# GLOBAL CONFIG (to be used later in Step - 5&7)
# ============================================================

columns_to_drop = [
    "Mapping Description", "Mapping Group", "Taskforce Person",
    "Content Reference", "Q Type", "Group Quanti?", "Question Options"
]

## Step 1: Load Data & Mapping

In [3]:
# ============================================================
# STEP 1 — MASTER LOADER PIPELINE (Modular & Clean)
# ============================================================

# ----------------------------------------------------
# Clean and standardize key metadata question wording
# ----------------------------------------------------
def clean_question_text(text):
    """Standardize key metadata questions so naming stays consistent."""
    replacements = {
        "What country is your organization headquartered in? If you are responding in a personal capacity, select the country where you are based.": "Country",

        "If you are a Corporate, Financial Institution, or Professional Service & Consultancy:What sector does your company operate in?": "Sector",

        "If you are a Corporate, Financial Institution, or Professional Service & Consultancy:What is your company’s SBTi status?": "SBTi Status",

        "If you are a Corporate, Financial Institution, or Professional Service & Consultancy:Select the range that best represents your total number of full-time employees in your most recent reporting year.": "Employees MRY",

        "If you are a Corporate, Financial Institution, or Professional Service & Consultancy:What was your company’s net annual turnover in the most recent reporting year(revenue)?": "Annual Turnover MRY",
    }
    return replacements.get(text, text)


# ----------------------------------------------------
# Identify boolean-like columns (values only 0/1)
# ----------------------------------------------------
def identify_boolean_columns(df):
    bool_cols = [
        col for col in df.columns 
        if set(df[col].dropna().unique()).issubset({0, 1})
    ]
    print(f"🟦 Boolean-like columns: {len(bool_cols)}")
    return bool_cols


# ----------------------------------------------------
# Convert 0/1 → TRUE/FALSE
# ----------------------------------------------------
def convert_bool_columns(df, bool_cols):
    df[bool_cols] = df[bool_cols].applymap(
        lambda x: "TRUE" if x == 1 else ("FALSE" if x == 0 else x)
    )
    print("🟩 Converted boolean values to TRUE/FALSE")
    return df


# ----------------------------------------------------
# Load cleaned dataset
# ----------------------------------------------------
def load_cleaned_dataset(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Missing cleaned dataset: {path}")
    df = pd.read_excel(path)
    print(f"📄 Loaded cleaned dataset: {df.shape}")
    return df


# ----------------------------------------------------
# Load question mapping + clean metadata question text
# ----------------------------------------------------
def load_question_mapping(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ Mapping doc not found at: {path}")

    xl = pd.ExcelFile(path)
    df_map = pd.read_excel(xl, sheet_name="mapping_output")

    df_map.columns = [
        "Question ID", "Question", "OG Question Number", "Section", "Subsection", "Mapping Description",
        "Mapping Group", "Question Group", "Taskforce Person", "Content Reference",
        "Q Type", "Group Quanti?", "Question Options"
    ]

    # Apply your metadata-cleaning helper
    df_map["Question"] = df_map["Question"].apply(clean_question_text)

    print(f"📘 Loaded question mapping: {df_map.shape}")
    return df_map


# ----------------------------------------------------
# Validate metadata columns exist in dataset
# ----------------------------------------------------
def validate_metadata(df_cleaned):
    meta_cols = [
        "Response ID", "Country", "Sector", "SBTi Status",
        "Employees MRY", "Annual Turnover MRY",
        "Category A/B", "World Bank Income Group", "Region"
    ]

    missing = set(meta_cols) - set(df_cleaned.columns)
    if missing:
        print(f"⚠ Missing metadata columns: {missing}")
    else:
        print("✅ All metadata columns found")

    return meta_cols


# ----------------------------------------------------
# Save intermediate mapping file
# ----------------------------------------------------
def save_intermediate(question_map):
    os.makedirs("intermediate_checks", exist_ok=True)
    question_map.to_excel(
        "intermediate_checks/question_mapping_cleaned.xlsx", index=False
    )
    print("💾 Saved cleaned mapping")


# ----------------------------------------------------
# MASTER FUNCTION — runs full Step 1
# ----------------------------------------------------
def Step1_load_and_prepare(
    cleaned_path="PC2_cleaned_full_dataset_anonymized.xlsx",
    mapping_path="../input/PC2_analysis_mapping_doc.xlsx"
):
    print("========== START STEP 1 ==========")

    df = load_cleaned_dataset(cleaned_path)

    bool_cols = identify_boolean_columns(df)
    df = convert_bool_columns(df, bool_cols)

    qmap = load_question_mapping(mapping_path)

    meta_cols = validate_metadata(df)

    save_intermediate(qmap)

    print("========== STEP 1 COMPLETE ==========\n")
    return df, qmap, meta_cols


# ----------------------------------------------------
# Execute Step 1
# ----------------------------------------------------
df_cleaned, question_map, meta_cols = Step1_load_and_prepare()


========== START STEP 1 ==========
📄 Loaded cleaned dataset: (26, 431)
🟦 Boolean-like columns: 279
🟩 Converted boolean values to TRUE/FALSE
📘 Loaded question mapping: (441, 13)
✅ All metadata columns found
💾 Saved cleaned mapping
========== STEP 1 COMPLETE ==========



## Step 2: Assign question numbers to data columns

In [4]:
# ============================================================
# STEP 2 — Assign Question Numbers + Validate Mappings
# ============================================================

def Step2_assign_question_numbers(df_cleaned, question_map, meta_cols, removed_questions=None):
    print("========== START STEP 2: Assign Question IDs ==========")

    if removed_questions is None:
        removed_questions = []

    # --------------------------------------------------------
    # Create numeric ID for each column in df_cleaned
    # --------------------------------------------------------
    column_numbers = {
        col: idx + 1 for idx, col in enumerate(df_cleaned.columns)
    }

    column_map_df = (
        pd.DataFrame(column_numbers.items(), columns=["Question", "Question ID"])
    )

    print(f"🟦 Assigned Question IDs to {len(column_map_df)} columns")

    # --------------------------------------------------------
    # Merge with question_map
    # --------------------------------------------------------
    merged_map = pd.merge(
        column_map_df,
        question_map,
        on="Question",
        how="left"
    )

    print(f"🔗 Merged mapping: {merged_map.shape}")

    # --------------------------------------------------------
    # Create masks for valid + quanti questions
    # --------------------------------------------------------

    # Normalize groups
    qmap_group = question_map["Mapping Group"].astype(str).str.lower()
    qmap_qtype = question_map["Q Type"].astype(str).str.lower()

    # Mask for valid (non-metadata, non-excluded)
    mask_valid_questions = ~qmap_group.isin(["meta", "not included", "survey feedback"])

    # Mask for quanti questions
    mask_quanti_questions = (
        qmap_qtype.eq("quanti")
        & ~question_map["Question"].isin(removed_questions)
        & ~question_map["Question"].isin(meta_cols)
        & ~qmap_group.isin(["meta", "not included", "survey feedback"])
    )

    valid_question_columns = (
        question_map[mask_valid_questions]["Question"].dropna().unique().tolist()
    )

    quanti_question_columns = (
        question_map[mask_quanti_questions]["Question"].dropna().unique().tolist()
    )

    print(f"🟩 Valid questions: {len(valid_question_columns)}")
    print(f"🟨 Quanti questions: {len(quanti_question_columns)}")

    # --------------------------------------------------------
    # Validate that all mapping questions exist in df_cleaned
    # --------------------------------------------------------

    q_errors = [
        q for q in valid_question_columns
        if q not in df_cleaned.columns
    ]

    if q_errors:
        print(f"⚠️ WARNING: {len(q_errors)} mapped questions NOT in dataset")
        os.makedirs("intermediate_checks", exist_ok=True)
        pd.DataFrame(q_errors, columns=["Missing Questions"]).to_csv(
            "intermediate_checks/q_errors.csv", index=False
        )
        print("   → Saved missing question list to intermediate_checks/q_errors.csv")
    else:
        print("✅ All valid questions exist in df_cleaned")

    print("========== STEP 2 COMPLETE ==========\n")

    return (
        column_map_df,
        merged_map,
        valid_question_columns,
        quanti_question_columns,
        q_errors
    )


# ------------------------------------------------------------
# Execute Step 2
# ------------------------------------------------------------
column_map_df, merged_map, valid_questions, quanti_questions, q_errors = Step2_assign_question_numbers(
    df_cleaned=df_cleaned,
    question_map=question_map,
    meta_cols=meta_cols,
    removed_questions=[]
)


========== START STEP 2: Assign Question IDs ==========
🟦 Assigned Question IDs to 431 columns
🔗 Merged mapping: (437, 14)
🟩 Valid questions: 196
🟨 Quanti questions: 77
✅ All valid questions exist in df_cleaned
========== STEP 2 COMPLETE ==========



## Step 3: Transpose survey to long format with metadata

In [5]:
# ============================================================
# STEP 3 — Transpose to Long Format + Progress Log
# ============================================================

def Step3_transpose_long_format(
    df_cleaned,
    question_map,
    meta_cols,
    valid_question_columns,
    mask_valid_questions,
    output_dir="intermediate_checks"
):
    print("========== START STEP 3: Long Format Transformation ==========")

    os.makedirs(output_dir, exist_ok=True)

    # ------------------------------------------------------------
    # 1. Create long-format dataset
    # ------------------------------------------------------------
    df_long = df_cleaned[meta_cols + valid_question_columns].melt(
        id_vars=meta_cols,
        var_name="Question",
        value_name="Answer"
    ).copy()

    print(f"📄 Long format dataset created: {df_long.shape}")

    # ------------------------------------------------------------
    # 2. Merge long format with question_map
    # ------------------------------------------------------------
    valid_mapping = question_map[
        question_map["Question"].isin(valid_question_columns)
    ]

    df_merged = pd.merge(
        df_long,
        valid_mapping,
        on="Question",
        how="left"
    )

    # Remove "index" pseudo-row if it appears
    df_merged = df_merged[df_merged["Question"] != "index"]

    # Remove rows with missing answers
    df_merged = df_merged[df_merged["Answer"].notna()]

    print(f"🧹 Cleaned merged long dataset: {df_merged.shape}")

    # ------------------------------------------------------------
    # 3. Check for valid questions that never appear in df_merged
    # ------------------------------------------------------------
    missing_in_long = [
        q for q in valid_question_columns
        if q not in df_merged["Question"].unique()
    ]

    if missing_in_long:
        print(f"⚠️ {len(missing_in_long)} valid questions never appear in long format.")
        pd.DataFrame(missing_in_long, columns=["Missing Questions"]).to_csv(
            f"{output_dir}/missing_in_long_format.csv",
            index=False
        )

    # Save merged long-format for inspection
    df_merged.to_excel(f"{output_dir}/long_format_transposed.xlsx", index=False)
    print(f"💾 Saved long-format dataset to {output_dir}/long_format_transposed.xlsx")

    # ------------------------------------------------------------
    # 4. Build Question Group Progress Log
    # ------------------------------------------------------------
    question_map_log = question_map.copy()

    # Keep only valid (non-meta, non-excluded) mapping groups
    question_map_log = question_map_log[mask_valid_questions].copy()

    # One row per mapping group (not per question)
    question_map_log = question_map_log.drop_duplicates(
        subset=["Mapping Group"]
    )

    # Keep only relevant columns
    keep_cols = [
        "Mapping Group",
        "Taskforce Person",
        "Content Reference"
    ]

    question_map_log = question_map_log[keep_cols].copy()

    # Rename Content Reference → Topic Owner
    question_map_log.rename(
        columns={"Content Reference": "Topic Owner"},
        inplace=True
    )

    # ------------------------------------------------------------
    # 5. Add empty workflow / progress columns
    # ------------------------------------------------------------
    progress_cols = [
        "Group contains mixed quanti/quali? (y/n)",
        "Initial topic identification: complete? (y/n)",
        "Review by topic owner(s): complete? (y/n)",
        "Final topic list: complete? (y/n)",
        "Qualitative Stage 2: complete? (y/n)",
        "Quantitative analysis: complete? (y/n)"
    ]

    for col in progress_cols:
        question_map_log[col] = None

    # Save progress log
    out_path = f"{output_dir}/question_group_analysis_progress_log.xlsx"
    question_map_log.to_excel(out_path, index=False)

    print(f"💾 Saved progress log to {out_path}")
    print("========== STEP 3 COMPLETE ==========\n")

    return df_merged, question_map_log, missing_in_long


# ------------------------------------------------------------
# Execute Step 3
# ------------------------------------------------------------

merged_filtered, question_group_log, missing_in_long = Step3_transpose_long_format(
    df_cleaned=df_cleaned,
    question_map=question_map,
    meta_cols=meta_cols,
    valid_question_columns=valid_questions,
    mask_valid_questions=question_map["Mapping Group"]
        .astype(str).str.lower()
        .isin(["meta", "not included", "survey feedback"])
        .__invert__()
)


========== START STEP 3: Long Format Transformation ==========
📄 Long format dataset created: (5096, 11)
🧹 Cleaned merged long dataset: (2050, 23)
💾 Saved long-format dataset to intermediate_checks/long_format_transposed.xlsx
💾 Saved progress log to intermediate_checks/question_group_analysis_progress_log.xlsx
========== STEP 3 COMPLETE ==========



# Step 4: Count valid responses per question

In [6]:
# ============================================================
# STEP 4 — Valid Response Counts + Crosstabs + Graphs
# ============================================================

def Step4_generate_summary_tables(merged_filtered, df_cleaned):
    print("========== START STEP 4: Response Summary ==========")

    # ------------------------------------------------------------
    # 1. Base Summary Table
    # ------------------------------------------------------------
    summary_table = (
        merged_filtered.groupby(["Question ID", "Question", "Q Type"])
        .agg(valid_responses=("Answer", "count"))
        .reset_index()
        .sort_values("Question ID")
    )

    summary_table["total_responses"] = len(df_cleaned)
    summary_table["response_rate"] = (
        summary_table["valid_responses"] / summary_table["total_responses"] * 100
    )

    summary_table.to_csv("valid_responses_count.csv", index=False)

    # ------------------------------------------------------------
    # 2. Quali + Quanti Split
    # ------------------------------------------------------------
    df_quali = summary_table[summary_table["Q Type"] == "quali"].copy()
    df_quali[">200"] = df_quali["valid_responses"] > 200
    df_quali.to_csv("quali_valid_responses_count.csv", index=False)

    df_quanti = summary_table[summary_table["Q Type"] == "quanti"].copy()
    df_quanti.to_csv("quanti_valid_responses_count.csv", index=False)

    # ------------------------------------------------------------
    # 3. Histogram
    # ------------------------------------------------------------
    plt.figure(figsize=(14, 6))
    plt.hist(
        [
            summary_table[summary_table["Q Type"] == "quali"]["valid_responses"],
            summary_table[summary_table["Q Type"] == "quanti"]["valid_responses"],
        ],
        bins=50,
        label=["Qualitative", "Quantitative"],
        alpha=0.7,
    )
    plt.xlabel("Valid Response Count")
    plt.ylabel("Frequency")
    plt.title("Histogram of Valid Response Counts by Question Type")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------------
    # 4. World Bank Income Group Breakdown
    # ------------------------------------------------------------
    wb_order = ["High income", "Upper middle income", "Lower middle income"]

    summary_wb = (
        merged_filtered.groupby(
            ["Question ID", "Question", "Q Type", "World Bank Income Group"]
        )
        .agg(valid_responses=("Answer", "count"))
        .reset_index()
    )

    wb_pivot = (
        summary_wb.pivot(
            index="Question ID",
            columns="World Bank Income Group",
            values="valid_responses",
        )
        .fillna(0)
        .reindex(columns=wb_order)
    )

    wb_pivot["Total"] = wb_pivot.sum(axis=1)
    wb_pivot.T.to_excel("PC2_question_answers_by_wb_income_group.xlsx")
    wb_pivot.T.to_csv("PC2_question_answers_by_wb_income_group.csv")

    # ------------------------------------------------------------
    # 5. Region Breakdown
    # ------------------------------------------------------------
    region_order = [
        "Europe",
        "Northern America",
        "Asia",
        "Latin America and the Caribbean",
        "Africa",
        "Oceania",
        "MENA",
    ]

    summary_reg = (
        merged_filtered.groupby(
            ["Question ID", "Question", "Q Type", "Region"]
        )
        .agg(valid_responses=("Answer", "count"))
        .reset_index()
    )

    reg_pivot = (
        summary_reg.pivot(index="Question ID", columns="Region", values="valid_responses")
        .fillna(0)
        .reindex(columns=region_order)
    )

    reg_pivot["Total"] = reg_pivot.sum(axis=1)
    reg_pivot.T.to_excel("PC2_question_answers_by_region.xlsx")
    reg_pivot.T.to_csv("PC2_question_answers_by_region.csv")

    print("========== STEP 4 COMPLETE ==========\n")

    return summary_table, df_quali, df_quanti, wb_pivot, reg_pivot




# Step 5: Export transposed data

In [7]:
# ============================================================
# STEP 5 — Export Clean Transposed Dataset (One Answer Per Line)
# ============================================================

def Step5_export_transposed_dataset(merged_filtered):
    print("========== START STEP 5: Export Transposed Dataset ==========")

    # Work on a copy
    df_save = merged_filtered.copy()

    # Replace NaN with "N/A" for better readability outside Python
    df_save.replace(np.nan, "N/A", inplace=True)

    # Drop helper/mapping columns
    df_save.drop(columns_to_drop, axis=1, inplace=True)

    # Save export files
    df_save.to_excel("PC2_all_responses_meta_data_one_answer_per_line.xlsx", index=False)
    df_save.to_csv("PC2_all_responses_meta_data_one_answer_per_line.csv", index=False)

    print("💾 Saved: PC2_all_responses_meta_data_one_answer_per_line.xlsx / .csv")
    print("========== STEP 5 COMPLETE ==========\n")

    return df_save


# ------------------------------------------------------------
# Execute Step 5
# ------------------------------------------------------------
df_step5_export = Step5_export_transposed_dataset(merged_filtered)


========== START STEP 5: Export Transposed Dataset ==========
💾 Saved: PC2_all_responses_meta_data_one_answer_per_line.xlsx / .csv
========== STEP 5 COMPLETE ==========



# Step 6: Group questions by mapping group and divide into separate spreadsheets

In [8]:
# ============================================================
# STEP 6 — Export Grouped Datasets (3 Output Types)
# ============================================================

def create_safe_name(text):
    """
    Clean strings so they can safely be used in folder/file names.
    Replaces slashes, spaces, and commas with safe characters.
    """
    return str(text).replace("/", "-").replace(" ", "_").replace(",", "_")


# ------------------------------------------------------------
# A. Export by Taskforce Person → Mapping Group → Region
# ------------------------------------------------------------
def export_by_person_and_region(merged_filtered):
    """
    Export A:
    For each Taskforce Person → each Mapping Group → split outputs by Region.
    Produces one Excel workbook per (Person × Group) with multiple region tabs.
    """
    print("📁 Export A: Taskforce Person → Group → Region")

    # List of all taskforce owners assigned to any question
    taskforce_people = merged_filtered["Taskforce Person"].dropna().unique()

    for person in taskforce_people:

        # Create safe folder names
        safe_person = create_safe_name(person)
        person_folder = f"output/grouped_responses_by_region/{safe_person}"
        os.makedirs(person_folder, exist_ok=True)

        # Subset all responses belonging to this taskforce owner
        person_subset = merged_filtered[merged_filtered["Taskforce Person"] == person]

        # Loop over each mapping group assigned to that person
        for group in person_subset["Mapping Group"].dropna().unique():
            safe_group = create_safe_name(group)

            # Filter to one specific question group
            group_subset = person_subset[person_subset["Mapping Group"] == group].copy()
            group_subset.sort_values(["Question ID", "Response ID"], inplace=True)

            # ------------------------------------------------------------
            # Split into quantitative and qualitative responses
            # ------------------------------------------------------------
            quant_df = group_subset[group_subset["Q Type"] == "quanti"].copy()
            quali_df = group_subset[group_subset["Q Type"] == "quali"].copy()

            # Create label maps so columns appear as "ID - Question text"
            quant_labels = {
                row["Question"]: f"{row['Question ID']} - {row['Question']}"
                for _, row in quant_df.drop_duplicates(subset=["Question"]).iterrows()
            }

            quali_labels = {
                row["Question"]: f"{row['Question ID']} - {row['Question']}"
                for _, row in quali_df.drop_duplicates(subset=["Question"]).iterrows()
            }

            # ------------------------------------------------------------
            # Pivot quantitative and qualitative into wide formats
            # ------------------------------------------------------------
            quant_pivot = quant_df.pivot(
                index="Response ID", columns="Question", values="Answer"
            ).reset_index()
            quant_pivot.rename(columns=quant_labels, inplace=True)

            quali_pivot = quali_df.pivot(
                index="Response ID", columns="Question", values="Answer"
            ).reset_index()
            quali_pivot.rename(columns=quali_labels, inplace=True)

            # ------------------------------------------------------------
            # Merge quant + quali so each row contains all answers for that respondent
            # ------------------------------------------------------------
            if len(quant_df) > 0:
                merged = quant_df.merge(quali_pivot, on="Response ID", how="left")
            else:
                merged = quali_df.merge(quant_pivot, on="Response ID", how="left")

            # ------------------------------------------------------------
            # Export to a multi-sheet Excel workbook split by Region
            # ------------------------------------------------------------
            output_path = f"{person_folder}/Question_Group_{safe_group}.xlsx"
            regions = merged["Region"].dropna().unique()

            with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:

                # Always include the full dataset ("Overall")
                merged.to_excel(writer, sheet_name="Overall", index=False)

                # Create one sheet per region
                for region in regions:
                    region_subset = merged[merged["Region"] == region]
                    sheet_name = create_safe_name(region)[:31]  # Excel limit
                    region_subset.to_excel(writer, sheet_name=sheet_name, index=False)

            print(f"   ✓ Saved: {output_path}")


In [9]:
# ------------------------------------------------------------
# B. Export by Question Group → Region (no taskforce split)
# ------------------------------------------------------------
def export_by_question_group(merged_filtered):
    """
    Export B:
    For each Mapping Group (question group), create a folder and export:
      • One combined worksheet ("Overall")
      • One sheet per Region
    This version does NOT split by taskforce person.
    """
    print("📁 Export B: Mapping Group → Region")

    # Loop over each question group defined in mapping
    for group in merged_filtered["Mapping Group"].dropna().unique():

        # Safe folder + file names
        safe_group = create_safe_name(group)
        group_folder = f"output/split_datasets_by_question_group/Question_Group_{safe_group}"
        os.makedirs(group_folder, exist_ok=True)

        # Filter responses belonging to this mapping group
        subset = merged_filtered[merged_filtered["Mapping Group"] == group].copy()
        subset.sort_values(["Question ID", "Response ID"], inplace=True)

        # ------------------------------------------------------------
        # Split into quantitative and qualitative
        # ------------------------------------------------------------
        quant_df = subset[subset["Q Type"] == "quanti"].copy()
        quali_df = subset[subset["Q Type"] == "quali"].copy()

        # Create column label maps for readability
        quant_labels = {
            row["Question"]: f"{row['Question ID']} - {row['Question']}"
            for _, row in quant_df.drop_duplicates(subset=["Question"]).iterrows()
        }

        quali_labels = {
            row["Question"]: f"{row['Question ID']} - {row['Question']}"
            for _, row in quali_df.drop_duplicates(subset=["Question"]).iterrows()
        }

        # Pivot quantitative and qualitative
        quant_pivot = quant_df.pivot(
            index="Response ID", columns="Question", values="Answer"
        ).reset_index()
        quant_pivot.rename(columns=quant_labels, inplace=True)

        quali_pivot = quali_df.pivot(
            index="Response ID", columns="Question", values="Answer"
        ).reset_index()
        quali_pivot.rename(columns=quali_labels, inplace=True)

        # Merge quanti + quali (same logic as original code)
        if len(quant_df) > 0:
            merged = quant_df.merge(quali_pivot, on="Response ID", how="left")
        else:
            merged = quali_df.merge(quant_pivot, on="Response ID", how="left")

        # Clean dataset: replace missing answers with "N/A"
        merged.replace(np.nan, "N/A", inplace=True)

        # Output path
        output_path = (
            f"{group_folder}/Question_Group_{safe_group}_Responses_By_Region.xlsx"
        )

        # Predefined region order
        regions = [
            "Europe", "Northern America", "Asia",
            "Latin America and the Caribbean",
            "Africa", "Oceania", "MENA"
        ]

        # ------------------------------------------------------------
        # Write multi-sheet workbook: Overall + each Region
        # ------------------------------------------------------------
        with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:

            merged.to_excel(writer, sheet_name="Overall", index=False)

            for region in regions:
                region_subset = merged[merged["Region"] == region]
                sheet_name = create_safe_name(region)[:31]  # Excel sheet name limit
                region_subset.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"   ✓ Saved: {output_path}")


In [10]:
# ------------------------------------------------------------
# C. Export by Taskforce Person → Group (no region split)
# ------------------------------------------------------------
def export_by_person(merged_filtered):
    """
    Export C:
    For each Taskforce Person → each Mapping Group:
      • Export one Excel file containing:
           - A Qualitative sheet
           - A Quantitative sheet
    No region-level splitting.
    """
    print("📁 Export C: Taskforce Person → Group (No Region Split)")

    taskforce_people = merged_filtered["Taskforce Person"].dropna().unique()

    for person in taskforce_people:
        safe_person = create_safe_name(person)
        person_folder = f"output/grouped_responses/{safe_person}"
        os.makedirs(person_folder, exist_ok=True)

        # Subset all responses assigned to this person
        person_subset = merged_filtered[merged_filtered["Taskforce Person"] == person].copy()

        for group in person_subset["Mapping Group"].dropna().unique():
            safe_group = create_safe_name(group)

            # Filter for one specific mapping group
            subset = person_subset[person_subset["Mapping Group"] == group].copy()
            subset.sort_values(["Question ID", "Response ID"], inplace=True)

            # Split quanti/quali
            quant_df = subset[subset["Q Type"] == "quanti"].copy()
            quali_df = subset[subset["Q Type"] == "quali"].copy()

            # Label map for quant
            quant_labels = {
                row["Question"]: f"{row['Question ID']} - {row['Question']}"
                for _, row in quant_df.drop_duplicates(subset=["Question"]).iterrows()
            }

            # Pivot quantitative data
            quant_pivot = quant_df.pivot(
                index="Response ID", columns="Question", values="Answer"
            ).reset_index()
            quant_pivot.rename(columns=quant_labels, inplace=True)

            # Merge qualitative + quantitative (original logic)
            merged = quali_df.merge(quant_pivot, on="Response ID", how="left")

            # Output path
            output_path = f"{person_folder}/Question_Group_{safe_group}.xlsx"

            # ------------------------------------------------------------
            # Write Excel: Qualitative tab + Quantitative tab
            # ------------------------------------------------------------
            with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
                merged.to_excel(writer, sheet_name="Qualitative", index=False)
                quant_df.to_excel(writer, sheet_name="Quantitative", index=False)

            print(f"   ✓ Saved: {output_path}")


In [11]:
# ------------------------------------------------------------
# MASTER FUNCTION — Runs Step 6 (all 3 output types)
# ------------------------------------------------------------
def Step6_generate_grouped_outputs(merged_filtered):
    """
    Runs all Step 6 exports:
      A. Taskforce Person → Group → Region
      B. Question Group → Region
      C. Taskforce Person → Group (no region split)

    This does NOT modify logic from the original notebook.
    """
    print("========== START STEP 6: Grouped Exports ==========")

    # Check required columns — same logic as original code
    required_cols = {
        "Mapping Group", "Question ID", "Response ID",
        "Taskforce Person", "Q Type", "Question", "Answer", "Region"
    }

    missing = required_cols - set(merged_filtered.columns)
    if missing:
        raise ValueError(f"Missing required columns for Step 6: {missing}")
    else:
        print("✅ All required columns present for Step 6")

    # Run all three export blocks
    export_by_person_and_region(merged_filtered)
    export_by_question_group(merged_filtered)
    export_by_person(merged_filtered)

    print("========== STEP 6 COMPLETE ==========\n")


## Step 7: Export all quantitative questions onto one separate sheet

In [12]:
# ============================================================
# STEP 7 — Export ALL Quantitative Questions (One Sheet)
# ============================================================

def Step7_export_quanti_only(
    df_cleaned,
    question_map,
    meta_cols,
    quanti_question_columns,
    columns_to_drop,
    output_dir="output"
):
    print("========== START STEP 7: Export ALL Quantitative Questions ==========")

    os.makedirs(output_dir, exist_ok=True)

    # --------------------------------------------------------
    # 1. Filter mapping down to only quantitative questions
    # --------------------------------------------------------
    quanti_mapping = question_map[
        question_map["Question"].isin(quanti_question_columns)
    ].copy()

    # --------------------------------------------------------
    # 2. Melt to long format (one row per answer)
    # --------------------------------------------------------
    df_long = (
        df_cleaned[meta_cols + quanti_question_columns]
        .reset_index()
        .melt(id_vars=meta_cols, var_name="Question", value_name="Answer")
    )

    # --------------------------------------------------------
    # 3. Merge with mapping doc
    # --------------------------------------------------------
    quanti_filtered = pd.merge(df_long, quanti_mapping, on="Question", how="left")

    # Remove non-answers and useless “index” dummy column
    quanti_filtered = quanti_filtered[
        (quanti_filtered["Question"] != "index") &
        (quanti_filtered["Answer"].notna())
    ]

    print(f"📄 Final quantitative long-format shape: {quanti_filtered.shape}")

    # --------------------------------------------------------
    # 4. Save FULL version including mapping columns
    # --------------------------------------------------------
    quanti_filtered.to_excel(
        f"{output_dir}/PC2_all_quantitative_responses_one_answer_per_line_for_analysis.xlsx",
        index=False
    )
    quanti_filtered.to_csv(
        f"{output_dir}/PC2_all_quantitative_responses_one_answer_per_line_for_analysis.csv",
        index=False
    )

    print("💾 Saved quantitative dataset WITH mapping (analysis version)")

    # --------------------------------------------------------
    # 5. Drop mapping columns (in-place)
    # --------------------------------------------------------
    quanti_filtered.drop(columns_to_drop, axis=1, inplace=True)

    # --------------------------------------------------------
    # 6. Save clean version (only meta + Question + Answer)
    # --------------------------------------------------------
    quanti_filtered.to_excel(
        f"{output_dir}/PC2_all_quantitative_responses_one_answer_per_line.xlsx",
        index=False
    )
    quanti_filtered.to_csv(
        f"{output_dir}/PC2_all_quantitative_responses_one_answer_per_line.csv",
        index=False
    )

    print("💾 Saved quantitative dataset CLEANED (no mapping columns)")
    print("========== STEP 7 COMPLETE ==========\n")

    return quanti_filtered

In [13]:
# ------------------------------------------------------------
# Execute Step 7
# ------------------------------------------------------------

quanti_output = Step7_export_quanti_only(
    df_cleaned=df_cleaned,
    question_map=question_map,
    meta_cols=meta_cols,
    quanti_question_columns=quanti_questions,   # from Step 2
    columns_to_drop=columns_to_drop,            # from Step 5
    output_dir="output"
)


========== START STEP 7: Export ALL Quantitative Questions ==========
📄 Final quantitative long-format shape: (1108, 23)
💾 Saved quantitative dataset WITH mapping (analysis version)
💾 Saved quantitative dataset CLEANED (no mapping columns)
========== STEP 7 COMPLETE ==========

